In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import gc

In [2]:
# Reading files

# Read in the data
# 2018
ela_2018 = pd.read_csv(
    r"C:\Users\ericr\Desktop\IDS 701 - Unifying Data Science\uds final project\Data-20230420T224029Z-001\Data\2018\Annual EM ELA.csv"
)
math_2018 = pd.read_csv(
    r"C:\Users\ericr\Desktop\IDS 701 - Unifying Data Science\uds final project\Data-20230420T224029Z-001\Data\2018\Annual EM Math.csv"
)
science_2018 = pd.read_csv(
    r"C:\Users\ericr\Desktop\IDS 701 - Unifying Data Science\uds final project\Data-20230420T224029Z-001\Data\2018\Annual EM SCIENCE.csv"
)



C:\Users\ericr\AppData\Local\Temp\ipykernel_6576\1614234590.py:5: DtypeWarning: Columns (15,16,17,18) have mixed types. Specify dtype option on import or set low_memory=False.
  ela_2018 = pd.read_csv(
C:\Users\ericr\AppData\Local\Temp\ipykernel_6576\1614234590.py:8: DtypeWarning: Columns (15,16,17,18) have mixed types. Specify dtype option on import or set low_memory=False.
  math_2018 = pd.read_csv(
C:\Users\ericr\AppData\Local\Temp\ipykernel_6576\1614234590.py:11: DtypeWarning: Columns (15,16,17,18) have mixed types. Specify dtype option on import or set low_memory=False.
  science_2018 = pd.read_csv(


In [3]:
# Checking types
ela_2018.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 654694 entries, 0 to 654693
Data columns (total 21 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   ENTITY_CD           654694 non-null  int64 
 1   ENTITY_NAME         654694 non-null  object
 2   YEAR                654694 non-null  int64 
 3   ASSESSMENT_NAME     654694 non-null  object
 4   SUBGROUP_NAME       654694 non-null  object
 5   NUM_TESTED          654694 non-null  int64 
 6   NOT_TESTED          654694 non-null  int64 
 7   LEVEL1_COUNT        654694 non-null  object
 8   LEVEL1_%TESTED      654694 non-null  object
 9   LEVEL2_COUNT        654694 non-null  object
 10  LEVEL2_%TESTED      654694 non-null  object
 11  LEVEL3_COUNT        654694 non-null  object
 12  LEVEL3_%TESTED      654694 non-null  object
 13  LEVEL4_COUNT        654694 non-null  object
 14  LEVEL4_%TESTED      654694 non-null  object
 15  LEVEL5_COUNT        304561 non-null  object
 16  LE

In [4]:
ela_2018["NUM_TESTED"].isnull().sum()

0

In [5]:
# Our columns of interest
Team_9_keep_cols = ["NUM_TESTED", "NUM_PROF", "TOTAL_SCALE_SCORES", "MEAN_SCORE"]
subgroup_9_keep_cols = [
    "Hispanic or Latino",
    "Black or African American",
    "American Indian or Alaska Native",
    "Asian or Native Hawaiian/Other Pacific Islander",
    "Multiracial",
    "White",
]

In [6]:
# Scouting racial groups
ela_2018.SUBGROUP_NAME.unique()

array(['Hispanic or Latino', 'Homeless', 'Male',
       'Non-English Language Learners', 'Not Economically Disadvantaged',
       'American Indian or Alaska Native',
       'Asian or Native Hawaiian/Other Pacific Islander',
       'Black or African American', 'Economically Disadvantaged',
       'English Language Learners', 'Female', 'General Education',
       'Multiracial', 'All Students', 'Not Migrant',
       'Students with Disabilities', 'White', 'Migrant',
       'Small Group Total', 'American Indian or Alaska Native Female',
       'American Indian or Alaska Native Male',
       'Asian or Native Hawaiian/Other Pacific Islander Female',
       'Asian or Native Hawaiian/Other Pacific Islander Male',
       'Black or African American Female',
       'Black or African American Male', 'Hispanic or Latino Female',
       'Hispanic or Latino Male', 'Multiracial Female',
       'Multiracial Male', 'White Female', 'White Male', 'Not Homeless',
       'Not in Foster Care', 'Parent in Arme

In [7]:



for col in Team_9_keep_cols:

    for val in ela_2018[col].unique():
        
        try:
            
            int(val)

            if int(val) <= 0:
                print(col, val)
            
        except:
            print(col, val)

NUM_PROF 0
NUM_PROF s
NUM_PROF nan
TOTAL_SCALE_SCORES s
TOTAL_SCALE_SCORES 00
MEAN_SCORE s
MEAN_SCORE 00
MEAN_SCORE nan


In [8]:


def sanity_check_entity_cd(ds):
    
    for i in ds.ENTITY_CD.unique():
        try: 
            int(i)
        except:
            print(i)

In [9]:
def race_df_maker(
    df: pd.DataFrame,
    base_cols: list[str],
    subgroups: list[str],
    keep_cols: list[str] = [],
    impute: bool = False,
    race_refactor: bool = False,
    show_refactor : bool = False,
    check : bool = False,
) -> pd.DataFrame:
    """Takes a our base dataframe and stacks the subgroups horizontally
    via a merge.
    """

    copy_df = df.copy()  # to contain all the core records

    copy_df["Subject"] = copy_df["ASSESSMENT_NAME"].str.extract(
        r"([A-Za-z]*).*", expand=False
    )

    if race_refactor:

        for col in keep_cols:
            try:
                copy_df[col].fillna(0, inplace=True)
            except:
                print(col, "fillna failed")

            try:
                copy_df[copy_df[col] == "nan"] = 0
            except:
                print(col, "string nan replacement failed")   
                pass
                 
            try:
                copy_df[copy_df[col] == "s"] = 0
            except:
                print(col, "s replacement failed")

            try:
                copy_df[copy_df[col] == "00"] = 0
            except:
                print(col, "00 replacement failed")
                raise ValueError("Processing error")
            
            copy_df[col] = copy_df[col].astype(float)

    else:
        copy_df[copy_df.MEAN_SCORE == "s"] = np.nan

    if impute:
        copy_df["MEAN_SCORE"] = copy_df["MEAN_SCORE"].astype(float)

        copy_df["MEAN_SCORE"].fillna(
            copy_df.groupby(["YEAR", "ENTITY_CD", "Subject"])["MEAN_SCORE"].transform(
                "mean"
            ),
            inplace=True,
        )

    base_cols.append("Subject")

    if keep_cols != []:
        new_cols = keep_cols + base_cols + ["SUBGROUP_NAME"]

    else:
        new_cols = base_cols + ["SUBGROUP_NAME"]

    base_df = copy_df[new_cols].copy()  # get the core records

    base_df.drop_duplicates(
        subset=base_cols, inplace=True, ignore_index=True
    )  # remove duplicates

    core_df = copy_df[new_cols].copy()  # get the core records

    # At this point, we have a df custom to merge with Team 9's base data
    # that contains all the schools,
    # But we still need the data by race

    # We will create a dictionary of dataframes

    for group in subgroups:
        # copy to avoid pointer issues

        group_df = core_df[core_df["SUBGROUP_NAME"] == group].copy()

        formatted_name = group.replace(" ", "_").lower()

        base_df = base_df.merge(
            group_df, on=base_cols, how="left", suffixes=("", f"_{formatted_name}")
        )

    # Removing the subgroup name columns

    ls_subgroup_cols = [
        col for col in base_df.columns if "subgroup_name" in col.lower()
    ]

    extra_cols = ["NUM_TESTED", "NUM_PROF", "TOTAL_SCALE_SCORES", "MEAN_SCORE"]

    base_df.drop(columns=ls_subgroup_cols + extra_cols, inplace=True)

    if race_refactor:

        for col in keep_cols:

            col_to_add = col + "_non_white"

            base_df[col_to_add] = 0

            for other_col in base_df.columns:

                if col in other_col and not ("white" in other_col):

                    base_df[col_to_add] += base_df[other_col].fillna(0)

            base_df[col_to_add] = base_df[col_to_add].astype(float)
            base_df[base_df[col_to_add] == 0] = np.nan

        
        if show_refactor:

            return base_df
        
        else:

            # Mean score fix

            see_dict = {"MEAN_SCORE" : []}
            for other_col in base_df.columns:
                
                # print(other_col, "MEAN_SCORE" in other_col)
                if "MEAN_SCORE" in other_col and not ("white" in other_col):
                    see_dict["MEAN_SCORE"].append(other_col)
                    # print("appended", other_col)

            # print(see_dict)
            mean_fix = base_df[see_dict["MEAN_SCORE"]].copy()
            mean_fix["MEAN_SCORE_non_white"] = mean_fix.mean(axis=1, skipna=True)
            base_df["MEAN_SCORE_non_white"] = mean_fix["MEAN_SCORE_non_white"]
            if check is True:
                return mean_fix
            
            
            for col in keep_cols:

                for other_col in base_df.columns:

                    if col in other_col and not ("white" in other_col):

                        base_df.drop(columns=other_col, inplace=True)

    return base_df

In [10]:
# testing the function
race_df_maker(
    ela_2018,
    ["ENTITY_CD", "YEAR", "ASSESSMENT_NAME"], # ENTITY_NAME
    subgroup_9_keep_cols, keep_cols=Team_9_keep_cols
).keys()

Index(['ENTITY_CD', 'YEAR', 'ASSESSMENT_NAME', 'Subject',
       'NUM_TESTED_hispanic_or_latino', 'NUM_PROF_hispanic_or_latino',
       'TOTAL_SCALE_SCORES_hispanic_or_latino',
       'MEAN_SCORE_hispanic_or_latino', 'NUM_TESTED_black_or_african_american',
       'NUM_PROF_black_or_african_american',
       'TOTAL_SCALE_SCORES_black_or_african_american',
       'MEAN_SCORE_black_or_african_american',
       'NUM_TESTED_american_indian_or_alaska_native',
       'NUM_PROF_american_indian_or_alaska_native',
       'TOTAL_SCALE_SCORES_american_indian_or_alaska_native',
       'MEAN_SCORE_american_indian_or_alaska_native',
       'NUM_TESTED_asian_or_native_hawaiian/other_pacific_islander',
       'NUM_PROF_asian_or_native_hawaiian/other_pacific_islander',
       'TOTAL_SCALE_SCORES_asian_or_native_hawaiian/other_pacific_islander',
       'MEAN_SCORE_asian_or_native_hawaiian/other_pacific_islander',
       'NUM_TESTED_multiracial', 'NUM_PROF_multiracial',
       'TOTAL_SCALE_SCORES_multirac

In [11]:
race_df_maker(
    ela_2018,
    ["ENTITY_CD", "YEAR", "ASSESSMENT_NAME"], # ENTITY_NAME
    subgroup_9_keep_cols, keep_cols=Team_9_keep_cols
).shape

(36740, 28)

In [12]:
x = race_df_maker(
    ela_2018,
    ["ENTITY_CD", "YEAR", "ASSESSMENT_NAME"], # ENTITY_NAME
    subgroup_9_keep_cols, keep_cols=Team_9_keep_cols
)

In [13]:
x.isnull().sum()

ENTITY_CD                                                                 1
YEAR                                                                      1
ASSESSMENT_NAME                                                           1
Subject                                                                   1
NUM_TESTED_hispanic_or_latino                                         20229
NUM_PROF_hispanic_or_latino                                           28298
TOTAL_SCALE_SCORES_hispanic_or_latino                                 20229
MEAN_SCORE_hispanic_or_latino                                         20229
NUM_TESTED_black_or_african_american                                  24739
NUM_PROF_black_or_african_american                                    30693
TOTAL_SCALE_SCORES_black_or_african_american                          24739
MEAN_SCORE_black_or_african_american                                  24739
NUM_TESTED_american_indian_or_alaska_native                           35656
NUM_PROF_ame

In [14]:
x

,ENTITY_CD,YEAR,ASSESSMENT_NAME,Subject,NUM_TESTED_hispanic_or_latino,NUM_PROF_hispanic_or_latino,TOTAL_SCALE_SCORES_hispanic_or_latino,MEAN_SCORE_hispanic_or_latino,NUM_TESTED_black_or_african_american,NUM_PROF_black_or_african_american,...,TOTAL_SCALE_SCORES_asian_or_native_hawaiian/other_pacific_islander,MEAN_SCORE_asian_or_native_hawaiian/other_pacific_islander,NUM_TESTED_multiracial,NUM_PROF_multiracial,TOTAL_SCALE_SCORES_multiracial,MEAN_SCORE_multiracial,NUM_TESTED_white,NUM_PROF_white,TOTAL_SCALE_SCORES_white,MEAN_SCORE_white
0,3.314000e+11,2018.0,ELA8,ELA,39.0,7,22910,587,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3.314000e+11,2018.0,ELA7,ELA,36.0,6,21346,593,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3.314000e+11,2018.0,ELA6,ELA,36.0,15,21466,596,12.0,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3.314000e+11,2018.0,ELA8,ELA,24.0,10,14345,598,14.0,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36735,6.808010e+11,2018.0,ELA5,ELA,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36736,6.808010e+11,2018.0,ELA3_8,ELA,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36737,6.808010e+11,2018.0,ELA6,ELA,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36738,6.808010e+11,2018.0,ELA3,ELA,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
x.isnull().sum()

ENTITY_CD                                                                 1
YEAR                                                                      1
ASSESSMENT_NAME                                                           1
Subject                                                                   1
NUM_TESTED_hispanic_or_latino                                         20229
NUM_PROF_hispanic_or_latino                                           28298
TOTAL_SCALE_SCORES_hispanic_or_latino                                 20229
MEAN_SCORE_hispanic_or_latino                                         20229
NUM_TESTED_black_or_african_american                                  24739
NUM_PROF_black_or_african_american                                    30693
TOTAL_SCALE_SCORES_black_or_african_american                          24739
MEAN_SCORE_black_or_african_american                                  24739
NUM_TESTED_american_indian_or_alaska_native                           35656
NUM_PROF_ame

In [16]:
y = race_df_maker(
    ela_2018,
    ["ENTITY_CD", "YEAR", "ASSESSMENT_NAME"], # ENTITY_NAME
    subgroup_9_keep_cols, keep_cols=Team_9_keep_cols,
    race_refactor=True,
    show_refactor=True,
)

y

,ENTITY_CD,YEAR,ASSESSMENT_NAME,Subject,NUM_TESTED_hispanic_or_latino,NUM_PROF_hispanic_or_latino,TOTAL_SCALE_SCORES_hispanic_or_latino,MEAN_SCORE_hispanic_or_latino,NUM_TESTED_black_or_african_american,NUM_PROF_black_or_african_american,...,TOTAL_SCALE_SCORES_multiracial,MEAN_SCORE_multiracial,NUM_TESTED_white,NUM_PROF_white,TOTAL_SCALE_SCORES_white,MEAN_SCORE_white,NUM_TESTED_non_white,NUM_PROF_non_white,TOTAL_SCALE_SCORES_non_white,MEAN_SCORE_non_white
0,3.314000e+11,2018.0,ELA8,ELA,39.0,7.0,22910.0,587.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,39.0,7.0,22910.0,587.0
1,3.314000e+11,2018.0,ELA7,ELA,36.0,6.0,21346.0,593.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,36.0,6.0,21346.0,593.0
2,3.314000e+11,2018.0,ELA6,ELA,36.0,15.0,21466.0,596.0,12.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,48.0,17.0,28589.0,1190.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3.314000e+11,2018.0,ELA8,ELA,24.0,10.0,14345.0,598.0,14.0,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,38.0,16.0,22729.0,1197.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32376,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32377,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32378,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32379,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
see_dict = {}
for col in Team_9_keep_cols:
    see_list = []   
    for other_col in y.columns:
        if col in other_col and (not ("white" in other_col) or "non_white" in other_col):
            see_list.append(other_col)
    see_dict[col] = see_list


In [18]:
y[see_dict['NUM_TESTED']]

,NUM_TESTED_hispanic_or_latino,NUM_TESTED_black_or_african_american,NUM_TESTED_american_indian_or_alaska_native,NUM_TESTED_asian_or_native_hawaiian/other_pacific_islander,NUM_TESTED_multiracial,NUM_TESTED_non_white
0,39.0,NaN,NaN,NaN,NaN,39.0
1,36.0,NaN,NaN,NaN,NaN,36.0
2,36.0,12.0,NaN,NaN,NaN,48.0
3,NaN,NaN,NaN,NaN,NaN,NaN
4,24.0,14.0,NaN,NaN,NaN,38.0
...,...,...,...,...,...,...
32376,NaN,NaN,NaN,NaN,NaN,NaN
32377,NaN,NaN,NaN,NaN,NaN,NaN
32378,NaN,NaN,NaN,NaN,NaN,NaN
32379,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
y[see_dict["NUM_PROF"]]

,NUM_PROF_hispanic_or_latino,NUM_PROF_black_or_african_american,NUM_PROF_american_indian_or_alaska_native,NUM_PROF_asian_or_native_hawaiian/other_pacific_islander,NUM_PROF_multiracial,NUM_PROF_non_white
0,7.0,NaN,NaN,NaN,NaN,7.0
1,6.0,NaN,NaN,NaN,NaN,6.0
2,15.0,2.0,NaN,NaN,NaN,17.0
3,NaN,NaN,NaN,NaN,NaN,NaN
4,10.0,6.0,NaN,NaN,NaN,16.0
...,...,...,...,...,...,...
32376,NaN,NaN,NaN,NaN,NaN,NaN
32377,NaN,NaN,NaN,NaN,NaN,NaN
32378,NaN,NaN,NaN,NaN,NaN,NaN
32379,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
y[see_dict["TOTAL_SCALE_SCORES"]]

,TOTAL_SCALE_SCORES_hispanic_or_latino,TOTAL_SCALE_SCORES_black_or_african_american,TOTAL_SCALE_SCORES_american_indian_or_alaska_native,TOTAL_SCALE_SCORES_asian_or_native_hawaiian/other_pacific_islander,TOTAL_SCALE_SCORES_multiracial,TOTAL_SCALE_SCORES_non_white
0,22910.0,NaN,NaN,NaN,NaN,22910.0
1,21346.0,NaN,NaN,NaN,NaN,21346.0
2,21466.0,7123.0,NaN,NaN,NaN,28589.0
3,NaN,NaN,NaN,NaN,NaN,NaN
4,14345.0,8384.0,NaN,NaN,NaN,22729.0
...,...,...,...,...,...,...
32376,NaN,NaN,NaN,NaN,NaN,NaN
32377,NaN,NaN,NaN,NaN,NaN,NaN
32378,NaN,NaN,NaN,NaN,NaN,NaN
32379,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
y[see_dict["MEAN_SCORE"]]

,MEAN_SCORE_hispanic_or_latino,MEAN_SCORE_black_or_african_american,MEAN_SCORE_american_indian_or_alaska_native,MEAN_SCORE_asian_or_native_hawaiian/other_pacific_islander,MEAN_SCORE_multiracial,MEAN_SCORE_non_white
0,587.0,NaN,NaN,NaN,NaN,587.0
1,593.0,NaN,NaN,NaN,NaN,593.0
2,596.0,594.0,NaN,NaN,NaN,1190.0
3,NaN,NaN,NaN,NaN,NaN,NaN
4,598.0,599.0,NaN,NaN,NaN,1197.0
...,...,...,...,...,...,...
32376,NaN,NaN,NaN,NaN,NaN,NaN
32377,NaN,NaN,NaN,NaN,NaN,NaN
32378,NaN,NaN,NaN,NaN,NaN,NaN
32379,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
y.isnull().sum()

ENTITY_CD                                                             21839
YEAR                                                                  21839
ASSESSMENT_NAME                                                       21839
Subject                                                               21839
NUM_TESTED_hispanic_or_latino                                         24044
NUM_PROF_hispanic_or_latino                                           24044
TOTAL_SCALE_SCORES_hispanic_or_latino                                 24044
MEAN_SCORE_hispanic_or_latino                                         24044
NUM_TESTED_black_or_african_american                                  26412
NUM_PROF_black_or_african_american                                    26412
TOTAL_SCALE_SCORES_black_or_african_american                          26412
MEAN_SCORE_black_or_african_american                                  26412
NUM_TESTED_american_indian_or_alaska_native                           31843
NUM_PROF_ame

In [23]:
z = race_df_maker(
    ela_2018,
    ["ENTITY_CD", "YEAR", "ASSESSMENT_NAME"], # ENTITY_NAME
    subgroup_9_keep_cols, keep_cols=Team_9_keep_cols,
    race_refactor=True,
)

z

,ENTITY_CD,YEAR,ASSESSMENT_NAME,Subject,NUM_TESTED_white,NUM_PROF_white,TOTAL_SCALE_SCORES_white,MEAN_SCORE_white,NUM_TESTED_non_white,NUM_PROF_non_white,TOTAL_SCALE_SCORES_non_white,MEAN_SCORE_non_white
0,3.314000e+11,2018.0,ELA8,ELA,NaN,NaN,NaN,NaN,39.0,7.0,22910.0,587.0
1,3.314000e+11,2018.0,ELA7,ELA,NaN,NaN,NaN,NaN,36.0,6.0,21346.0,593.0
2,3.314000e+11,2018.0,ELA6,ELA,NaN,NaN,NaN,NaN,48.0,17.0,28589.0,595.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3.314000e+11,2018.0,ELA8,ELA,NaN,NaN,NaN,NaN,38.0,16.0,22729.0,598.5
...,...,...,...,...,...,...,...,...,...,...,...,...
32376,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32377,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32378,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32379,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
race_df_maker(
    ela_2018,
    ["ENTITY_CD", "YEAR", "ASSESSMENT_NAME"], # ENTITY_NAME
    subgroup_9_keep_cols, keep_cols=Team_9_keep_cols,
    race_refactor=True, check=True,
)


,MEAN_SCORE_hispanic_or_latino,MEAN_SCORE_black_or_african_american,MEAN_SCORE_american_indian_or_alaska_native,MEAN_SCORE_asian_or_native_hawaiian/other_pacific_islander,MEAN_SCORE_multiracial,MEAN_SCORE_non_white
0,587.0,NaN,NaN,NaN,NaN,587.0
1,593.0,NaN,NaN,NaN,NaN,593.0
2,596.0,594.0,NaN,NaN,NaN,595.0
3,NaN,NaN,NaN,NaN,NaN,NaN
4,598.0,599.0,NaN,NaN,NaN,598.5
...,...,...,...,...,...,...
32376,NaN,NaN,NaN,NaN,NaN,NaN
32377,NaN,NaN,NaN,NaN,NaN,NaN
32378,NaN,NaN,NaN,NaN,NaN,NaN
32379,NaN,NaN,NaN,NaN,NaN,NaN


### 2018

In [25]:
# testing the function
for ds in [ela_2018, math_2018, science_2018]:
    sanity_check_entity_cd(ds)
    try: 
        race_df_maker(
            ds,
            ["ENTITY_CD", "YEAR", "ASSESSMENT_NAME"], # ENTITY_NAME
            subgroup_9_keep_cols, keep_cols=Team_9_keep_cols,
            race_refactor=True,
        )

    except Exception as e:
        print(e)
        print(ds.head())
        print(ds.shape)

In [26]:
final_ds = None

# CHANGE HERE

for ds in [ela_2018, math_2018, science_2018]:
    assert 2018 in ds.YEAR.unique()
    print(ds.YEAR.unique())

#     new_ds = race_df_maker(
#         ds,
#         ["ENTITY_CD", "YEAR", "ASSESSMENT_NAME"],  # ENTITY_NAME
#         subgroup_9_keep_cols,
#         keep_cols=Team_9_keep_cols,
#         race_refactor=True,
#     )

#     if final_ds is None:
#         final_ds = new_ds.copy()

#     else:
#         final_ds = pd.concat([final_ds, new_ds], ignore_index=True)

[2018 2017]
[2017 2018]
[2018 2017]


In [28]:

# sanity checks

# print("Shape of final_ds:", final_ds.shape)
# print("")

In [31]:
# final_ds.head(10)

In [30]:
# delete the dataframes to save memory
del ela_2018, math_2018, science_2018
gc.collect()

640

### 2019

In [32]:
# 2019

ela_2019 = pd.read_csv(
    r"C:\Users\ericr\Desktop\IDS 701 - Unifying Data Science\uds final project\Data-20230420T224029Z-001\Data\2019\Annual EM ELA.csv"
)
math_2019 = pd.read_csv(
    r"C:\Users\ericr\Desktop\IDS 701 - Unifying Data Science\uds final project\Data-20230420T224029Z-001\Data\2019\Annual EM MATH.csv"
)
science_2019 = pd.read_csv(
    r"C:\Users\ericr\Desktop\IDS 701 - Unifying Data Science\uds final project\Data-20230420T224029Z-001\Data\2019\Annual EM SCIENCE.csv"
)

# testing the function
for ds in [ela_2019, math_2019, science_2019]:
    sanity_check_entity_cd(ds)

    try: 
        race_df_maker(
            ds,
            ["ENTITY_CD", "YEAR", "ASSESSMENT_NAME"], # ENTITY_NAME
            subgroup_9_keep_cols, keep_cols=Team_9_keep_cols,
            race_refactor=True,
        )

    except Exception as e:
        print(e)
        print(ds.head())
        print(ds.shape)



In [33]:
for ds in [ela_2019, math_2019, science_2019]:

    assert 2019 in ds.YEAR.unique()
    print(ds.YEAR.unique())
    
    new_ds = race_df_maker(
        ds,
        ["ENTITY_CD", "YEAR", "ASSESSMENT_NAME"],  # ENTITY_NAME
        subgroup_9_keep_cols,
        keep_cols=Team_9_keep_cols,
        race_refactor=True,
    )

    if final_ds is None:

        final_ds = new_ds.copy()

    else:

        final_ds = pd.concat([final_ds, new_ds], ignore_index=True)

[2018 2019]
[2018 2019]
[2018 2019]


In [34]:

# sanity checks

print("Shape of final_ds:", final_ds.shape)
print("")

Shape of final_ds: (91606, 12)



In [35]:
final_ds.tail(10)

,ENTITY_CD,YEAR,ASSESSMENT_NAME,Subject,NUM_TESTED_white,NUM_PROF_white,TOTAL_SCALE_SCORES_white,MEAN_SCORE_white,NUM_TESTED_non_white,NUM_PROF_non_white,TOTAL_SCALE_SCORES_non_white,MEAN_SCORE_non_white
91596,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
91597,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
91598,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
91599,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
91600,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
91601,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
91602,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
91603,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
91604,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
91605,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
# delete the dataframes to save memory
del ela_2019, math_2019, science_2019
gc.collect()

0

### 2020 does not exist

### 2021

In [37]:



# 2020 is unobserved

# 2021
ela_2021 = pd.read_csv(
    r"C:\Users\ericr\Desktop\IDS 701 - Unifying Data Science\uds final project\Data-20230420T224029Z-001\Data\2021\Annual EM ELA.csv"
)
math_2021 = pd.read_csv(
    r"C:\Users\ericr\Desktop\IDS 701 - Unifying Data Science\uds final project\Data-20230420T224029Z-001\Data\2021\Annual EM Math.csv"
)
science_2021 = pd.read_csv(
    r"C:\Users\ericr\Desktop\IDS 701 - Unifying Data Science\uds final project\Data-20230420T224029Z-001\Data\2021\Annual EM SCIENCE.csv"
)


# testing the function

for ds in [ela_2021, math_2021, science_2021]:
    
    sanity_check_entity_cd(ds)

    try: 
        race_df_maker(
            ds,
            ["ENTITY_CD", "YEAR", "ASSESSMENT_NAME"], # ENTITY_NAME
            subgroup_9_keep_cols, keep_cols=Team_9_keep_cols,
            race_refactor=True,
        )

    except Exception as e:
        print(ds.YEAR.unique(), ds.ASSESSMENT_NAME.unique())
        print(e)
        print(ds.head())
        print(ds.shape)
        continue

C:\Users\ericr\AppData\Local\Temp\ipykernel_6576\684444587.py:7: DtypeWarning: Columns (19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  math_2021 = pd.read_csv(


TOTAL_SCALE_SCORES fillna failed
TOTAL_SCALE_SCORES string nan replacement failed
TOTAL_SCALE_SCORES s replacement failed
TOTAL_SCALE_SCORES 00 replacement failed
[2021] ['ELA3' 'ELA4' 'ELA5' 'ELA6' 'ELA7' 'ELA8' 'ELA3_8']
Processing error
   INSTITUTION_ID     ENTITY_CD         ENTITY_NAME  YEAR ASSESSMENT_NAME   
0    8.000001e+11  111111111111  All Public Schools  2021            ELA3  \
1    8.000001e+11  111111111111  All Public Schools  2021            ELA3   
2    8.000001e+11  111111111111  All Public Schools  2021            ELA3   
3    8.000001e+11  111111111111  All Public Schools  2021            ELA3   
4    8.000001e+11  111111111111  All Public Schools  2021            ELA3   

                      SUBGROUP_NAME  TOTAL_COUNT  NOT_TESTED  PCT_NOT_TESTED   
0                      All Students       192560       99642              52  \
1                            Female        93485       47472              51   
2                              Male        99075       52

In [38]:
math_2021.keys()

Index(['INSTITUTION_ID', 'ENTITY_CD', 'ENTITY_NAME', 'YEAR', 'ASSESSMENT_NAME',
       'SUBGROUP_NAME', 'TOTAL_COUNT', 'NOT_TESTED', 'PCT_NOT_TESTED',
       'NUM_TESTED', 'PCT_TESTED', 'LEVEL1_COUNT', 'LEVEL1_%TESTED',
       'LEVEL2_COUNT', 'LEVEL2_%TESTED', 'LEVEL3_COUNT', 'LEVEL3_%TESTED',
       'LEVEL4_COUNT', 'LEVEL4_%TESTED', 'LEVEL5_COUNT', 'LEVEL5_%TESTED',
       'NUM_PROF', 'PER_PROF', 'TOTAL_EXEMPT', 'NUM_EXEMPT_NTEST',
       'PCT_EXEMPT_NTEST', 'NUM_EXEMPT_TEST', 'PCT_EXEMPT_TEST'],
      dtype='object')

>2021 has no mean scores. The message propagates across all 3 subjects. 

In [39]:
# delete the dataframes to save memory
del ela_2021, math_2021, science_2021
gc.collect()

66

### 2022

In [40]:


# 2022

ela_2022 = pd.read_csv(
    r"C:\Users\ericr\Desktop\IDS 701 - Unifying Data Science\uds final project\Data-20230420T224029Z-001\Data\2022\Annual EM ELA.csv"
)
math_2022 = pd.read_csv(
    r"C:\Users\ericr\Desktop\IDS 701 - Unifying Data Science\uds final project\Data-20230420T224029Z-001\Data\2022\Annual EM Math.csv"
)
science_2022 = pd.read_csv(
    r"C:\Users\ericr\Desktop\IDS 701 - Unifying Data Science\uds final project\Data-20230420T224029Z-001\Data\2022\Annual EM SCIENCE.csv"
)

# testing the function
for ds in [ela_2022, math_2022, science_2022]:

    sanity_check_entity_cd(ds)

    try: 
        race_df_maker(
            ds,
            ["ENTITY_CD", "YEAR", "ASSESSMENT_NAME"], # ENTITY_NAME
            subgroup_9_keep_cols, keep_cols=Team_9_keep_cols,
            race_refactor=True,
        )

    except Exception as e:
        print(e)
        print(ds.head())
        print(ds.shape)

C:\Users\ericr\AppData\Local\Temp\ipykernel_6576\3543199631.py:6: DtypeWarning: Columns (19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  math_2022 = pd.read_csv(


In [41]:
for ds in [ela_2022, math_2022, science_2022]:

    assert 2022 in ds.YEAR.unique()
    print(ds.YEAR.value_counts())

    print(ds.YEAR.unique())
    
    new_ds = race_df_maker(
        ds,
        ["ENTITY_CD", "YEAR", "ASSESSMENT_NAME"],  # ENTITY_NAME
        subgroup_9_keep_cols,
        keep_cols=Team_9_keep_cols,
        race_refactor=True,
    )

    if final_ds is None:

        final_ds = new_ds.copy()

    else:

        final_ds = pd.concat([final_ds, new_ds], ignore_index=True)

YEAR
2022    321603
2021    319420
Name: count, dtype: int64
[2021 2022]
YEAR
2022    330819
2021    325779
Name: count, dtype: int64
[2021 2022]
YEAR
2022    112881
2021    111678
Name: count, dtype: int64
[2021 2022]


In [42]:
# sanity checks

print("Shape of final_ds:", final_ds.shape)
print("")
final_ds.tail(10)

Shape of final_ds: (215559, 12)



,ENTITY_CD,YEAR,ASSESSMENT_NAME,Subject,NUM_TESTED_white,NUM_PROF_white,TOTAL_SCALE_SCORES_white,MEAN_SCORE_white,NUM_TESTED_non_white,NUM_PROF_non_white,TOTAL_SCALE_SCORES_non_white,MEAN_SCORE_non_white
215549,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
215550,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
215551,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
215552,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
215553,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
215554,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
215555,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
215556,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
215557,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
215558,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [43]:
# delete the dataframes to save memory
del ela_2022, math_2022, science_2022

### Merging with Team 9's All Data CSV

In [44]:
all_data = pd.read_csv(r"C:\Users\ericr\Desktop\IDS 701 - Unifying Data Science\uds final project\unifying-data-science-2023-project-grupo9\final.csv")

In [45]:
# sanity checks
print("Shape of all_data:", all_data.shape)

Shape of all_data: (2933498, 4)


In [46]:

all_data.head()

,ENTITY_CD,YEAR,Subject,ASSESSMENT_NAME
0,1,2018,ELA,ELA5
1,1,2018,ELA,ELA3_8
2,1,2018,ELA,ELA3
3,1,2018,ELA,ELA7
4,1,2018,ELA,ELA8


In [47]:
full_data = pd.merge(all_data, final_ds, how="left", on=["ENTITY_CD", "YEAR", "Subject", "ASSESSMENT_NAME"])
full_data.head()

,ENTITY_CD,YEAR,Subject,ASSESSMENT_NAME,NUM_TESTED_white,NUM_PROF_white,TOTAL_SCALE_SCORES_white,MEAN_SCORE_white,NUM_TESTED_non_white,NUM_PROF_non_white,TOTAL_SCALE_SCORES_non_white,MEAN_SCORE_non_white
0,1,2018,ELA,ELA5,11063.0,6383.0,6745329.0,610.0,57376.0,19607.0,34364993.0,602.6
1,1,2018,ELA,ELA3_8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,2018,ELA,ELA3,11416.0,7869.0,6950213.0,609.0,55821.0,26167.0,33411345.0,602.0
3,1,2018,ELA,ELA7,10365.0,6680.0,6332714.0,611.0,54885.0,21132.0,32922142.0,603.4
4,1,2018,ELA,ELA8,9850.0,6707.0,6002917.0,609.0,53333.0,25335.0,32016776.0,603.2


In [48]:
full_data.keys()

Index(['ENTITY_CD', 'YEAR', 'Subject', 'ASSESSMENT_NAME', 'NUM_TESTED_white',
       'NUM_PROF_white', 'TOTAL_SCALE_SCORES_white', 'MEAN_SCORE_white',
       'NUM_TESTED_non_white', 'NUM_PROF_non_white',
       'TOTAL_SCALE_SCORES_non_white', 'MEAN_SCORE_non_white'],
      dtype='object')

In [49]:
full_data.to_csv(r"C:\Users\ericr\Desktop\IDS 701 - Unifying Data Science\uds final project\unifying-data-science-2023-project-grupo9\full_data_BinaryRace_Fixed.csv", index=False)

In [48]:
full_data.isnull().sum()

na_cols = full_data.isnull().sum()[full_data.isnull().sum() > 0].index

In [49]:
full_data.isnull().sum()


ENTITY_CD                             0
YEAR                                  0
Subject                               0
ASSESSMENT_NAME                       0
NUM_TESTED_white                2221634
NUM_PROF_white                  2221634
TOTAL_SCALE_SCORES_white        2221634
MEAN_SCORE_white                2221634
NUM_TESTED_non_white            1659153
NUM_PROF_non_white              1659153
TOTAL_SCALE_SCORES_non_white    1659153
MEAN_SCORE_non_white            1659153
dtype: int64

In [50]:
test_copy = full_data.copy()

In [51]:

# for col in na_cols:

#     test_copy[col].fillna(test_copy.groupby(["YEAR","ENTITY_CD","Subject", "ASSESSMENT_NAME"])[col].transform("median"), inplace=True)

In [52]:
test_copy.head()

,ENTITY_CD,YEAR,Subject,ASSESSMENT_NAME,NUM_TESTED_white,NUM_PROF_white,TOTAL_SCALE_SCORES_white,MEAN_SCORE_white,NUM_TESTED_non_white,NUM_PROF_non_white,TOTAL_SCALE_SCORES_non_white,MEAN_SCORE_non_white
0,1,2018,ELA,ELA5,11063.0,6383.0,6745329.0,610.0,57376.0,19607.0,34364993.0,602.6
1,1,2018,ELA,ELA5,11063.0,6383.0,6745329.0,610.0,57376.0,19607.0,34364993.0,602.6
2,1,2018,ELA,ELA3_8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,2018,ELA,ELA3,11416.0,7869.0,6950213.0,609.0,55821.0,26167.0,33411345.0,602.0
4,1,2018,ELA,ELA3,11416.0,7869.0,6950213.0,609.0,55821.0,26167.0,33411345.0,602.0


In [53]:
test_copy.isnull().sum()

ENTITY_CD                             0
YEAR                                  0
Subject                               0
ASSESSMENT_NAME                       0
NUM_TESTED_white                2221634
NUM_PROF_white                  2221634
TOTAL_SCALE_SCORES_white        2221634
MEAN_SCORE_white                2221634
NUM_TESTED_non_white            1659153
NUM_PROF_non_white              1659153
TOTAL_SCALE_SCORES_non_white    1659153
MEAN_SCORE_non_white            1659153
dtype: int64

In [54]:
full_data

full_data["MEAN_SCORE"].fillna(copy_df.groupby(["YEAR", "ENTITY_CD", "Subject"])["MEAN_SCORE"].transform("mean"), inplace=True)


KeyError: 'MEAN_SCORE'